In [29]:
import numpy as np
import os
import pandas as pd
from time import time
import pickle
from itertools import combinations

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectKBest,f_classif,chi2

#from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel

from sklearn.feature_selection import RFECV

In [2]:
def build_model():
    estimators = [
    ('rf', RandomForestClassifier(n_estimators=10, random_state=42)),
    ('svc', SVC(kernel = 'sigmoid',gamma='auto')),
    ('knc',KNeighborsClassifier(n_neighbors=3)),
    ('GNB',GaussianNB()),
    ('QDA',QuadraticDiscriminantAnalysis()),
    ('DTC',DecisionTreeClassifier())
    ]
    clf = StackingClassifier(
        estimators=estimators, final_estimator= SGDClassifier(), cv=10, #LogisticRegression
        n_jobs = 18
    )
    
    return clf

In [2]:
_file_name = 'Adjusted categorization - 1-5,16-30, 35-70.xlsx'

In [3]:
xl = pd.ExcelFile(_file_name)
xl.sheet_names

['H vs. 1-15',
 'H vs. Sc35-70',
 'H vs. Sc 16-34',
 'Multiclasses',
 'All categories',
 'VEG index',
 'spss Veg class1',
 'Spss veg class 2',
 'Spss veg class 3',
 'SPSS multi VEG adjusted']

In [4]:
full_data = pd.read_excel(_file_name, sheet_name='Multiclasses')
full_data.head()

,label,394.6,396.7,398.7,400.8,402.8,404.9,406.9,409,411,...,866.7,868.8,870.9,872.9,875,877,879.1,881.1,883.2,885.2
0,0,0.04827,0.04468,0.04008,0.03521,0.03030,0.02578,0.02212,0.01985,0.01826,...,0.3162,0.3168,0.3175,0.3180,0.3184,0.3187,0.3190,0.3194,0.3195,0.3194
1,0,0.05322,0.04898,0.04387,0.03805,0.03224,0.02683,0.02242,0.01950,0.01754,...,0.3379,0.3388,0.3393,0.3402,0.3408,0.3412,0.3415,0.3417,0.3419,0.3416
2,0,0.04710,0.04375,0.03963,0.03504,0.03022,0.02590,0.02229,0.01998,0.01833,...,0.3627,0.3634,0.3638,0.3643,0.3648,0.3651,0.3651,0.3649,0.3651,0.3648
3,0,0.04965,0.04648,0.04230,0.03775,0.03321,0.02890,0.02520,0.02265,0.02040,...,0.3373,0.3383,0.3390,0.3399,0.3406,0.3412,0.3419,0.3423,0.3429,0.3432
4,0,0.04562,0.04221,0.03784,0.03332,0.02895,0.02493,0.02176,0.02012,0.01829,...,0.2413,0.2420,0.2431,0.2438,0.2444,0.2452,0.2458,0.2463,0.2465,0.2471


In [45]:
str_cols = full_data.columns.astype(str)
full_data.columns = str_cols
cols = str_cols.tolist()
cols = cols[1:]

__input = full_data[cols]
label = full_data['label'].tolist()
label = np.asarray(label).reshape(-1,)

In [47]:
_input

,394.6,396.7,398.7,400.8,402.8,404.9,406.9,409,411,413.1,...,866.7,868.8,870.9,872.9,875,877,879.1,881.1,883.2,885.2
0,0.370963,0.405076,0.345565,0.396703,0.404955,0.373281,0.365420,0.326933,0.279768,0.268328,...,0.673293,0.672524,0.672910,0.672889,0.673541,0.673346,0.673178,0.673247,0.672903,0.672594
1,0.439866,0.475939,0.404747,0.457504,0.451167,0.402750,0.374617,0.315345,0.255203,0.278832,...,0.738454,0.738378,0.738029,0.739086,0.740202,0.740040,0.739665,0.738951,0.738793,0.737840
2,0.354677,0.389750,0.338538,0.393064,0.403049,0.376649,0.370632,0.331237,0.282156,0.289336,...,0.812924,0.812015,0.811214,0.810949,0.811624,0.810885,0.809403,0.807307,0.807036,0.806025
3,0.390173,0.434740,0.380231,0.451081,0.474273,0.460848,0.459841,0.419633,0.352781,0.367937,...,0.736652,0.736881,0.737133,0.738192,0.739607,0.740040,0.740847,0.740719,0.741734,0.742542
4,0.334076,0.364370,0.310587,0.356241,0.372797,0.349425,0.354384,0.335872,0.280792,0.291147,...,0.448381,0.448619,0.450668,0.451634,0.453323,0.455478,0.456872,0.457867,0.458172,0.460103
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2935,0.606069,0.626071,0.513117,0.574609,0.562887,0.541959,0.565297,0.604370,0.544524,0.574761,...,0.438772,0.439638,0.440511,0.440899,0.441419,0.443621,0.446529,0.449912,0.451700,0.453049
2936,0.578229,0.630356,0.539350,0.633483,0.633397,0.608476,0.586757,0.538487,0.483453,0.504491,...,0.353793,0.354028,0.354781,0.353829,0.354522,0.354992,0.355811,0.356806,0.358454,0.359882
2937,0.450167,0.525544,0.486727,0.618069,0.671987,0.697165,0.718271,0.672571,0.636984,0.650101,...,0.354093,0.355824,0.356872,0.359196,0.361367,0.362106,0.363198,0.362994,0.361984,0.360176
2938,0.485523,0.563777,0.514366,0.653607,0.720581,0.765366,0.800123,0.776196,0.743773,0.757317,...,0.394331,0.394139,0.395107,0.395575,0.396482,0.397380,0.397476,0.397171,0.397576,0.397502


In [46]:
scaler = MinMaxScaler()
_std = scaler.fit_transform(__input)
_input = pd.DataFrame(_std,columns=cols)

### Tree based feature selection

In [8]:
etc = ExtraTreesClassifier(n_estimators=200)
etc = etc.fit(_input, label)

In [64]:
etc.feature_importances_

array([0.00475482, 0.00484353, 0.00637623, 0.00849145, 0.0195514 ,
       0.01564198, 0.00731358, 0.00677097, 0.01219501, 0.00868838,
       0.00535836, 0.00400797, 0.00407264, 0.00483207, 0.00404343,
       0.00396532, 0.00348179, 0.00372575, 0.00334861, 0.00344157,
       0.0036709 , 0.00358186, 0.00403337, 0.0035024 , 0.00394557,
       0.00387732, 0.00384655, 0.00358605, 0.00397003, 0.00365523,
       0.00368488, 0.00374159, 0.00351681, 0.00357455, 0.00363041,
       0.00317823, 0.00318667, 0.00318389, 0.00344295, 0.00338372,
       0.00337333, 0.00324544, 0.00392854, 0.00354275, 0.00316823,
       0.00322568, 0.00349376, 0.00342628, 0.0032341 , 0.00314491,
       0.00274759, 0.00325994, 0.00325885, 0.00365836, 0.00313877,
       0.00343774, 0.00353453, 0.00387444, 0.00383852, 0.0037033 ,
       0.00352736, 0.0036081 , 0.00418425, 0.00358516, 0.00362783,
       0.00383498, 0.00443673, 0.00352319, 0.00348622, 0.00396532,
       0.00354332, 0.00379292, 0.00364864, 0.00369031, 0.00328

In [65]:
indices = np.argsort(etc.feature_importances_)[::-1]
indices

array([  4,   5, 179,   8, 180,   9,   3,   6, 178,   7,   2, 207, 161,
       145, 166, 177, 185,  10, 156, 159, 206, 204, 167, 148, 146, 132,
         1, 150,  13, 152,   0, 131, 176, 127, 155, 135, 149, 165, 143,
       175, 157, 133, 147, 184, 128, 123, 139,  66, 181, 125, 223, 136,
       208, 124,  95, 211, 187, 162, 130, 141, 192, 183, 151, 144, 205,
       214, 193, 163,  62, 215, 202, 226, 153, 137, 121, 225,  12, 237,
        14, 230,  22, 234, 134, 142, 196,  11, 218, 238,  28,  69,  15,
        81, 172, 203, 105, 235,  24, 194,  42, 239, 140, 107, 195,  25,
        57, 160,  76,  77, 228,  26, 188, 164,  58,  65, 120, 222, 182,
       224, 198,  71,  88, 112, 138, 191, 169, 122, 103, 201,  31, 216,
        78, 227,  17, 116, 154,  59, 199,  75,  73, 210,  30, 170,  20,
       200, 168,  53,  29, 186, 232,  72,  34,  64, 117, 171,  61, 101,
        27,  63,  21, 119,  33, 236, 111, 126, 190,  70,  43,  56,  60,
       115,  67, 213,  32,  23, 212, 189,  46, 118, 173,  68, 20

In [67]:
top_bands = list(_input.columns.values[indices])
top_bands[:10]

['402.8',
 '404.9',
 '762.1',
 '411',
 '764.1',
 '413.1',
 '400.8',
 '406.9',
 '760',
 '409']

In [9]:
sfm = SelectFromModel(etc)
X_new = sfm.fit_transform(_input, label)
X_new.shape  

(1483, 72)

In [45]:
# Model selected 93 bands
col_importance = []

for col, importance in zip(cols,etc.feature_importances_):
    item= dict()
    item['col'] = col
    item['importance'] = importance
    col_importance.append(item)

pd_col_imp = pd.DataFrame(col_importance)
_number = X_new.shape[1]
top_bands = pd_col_imp.sort_values(by=['importance'], ascending=False).head(_number)['col'].tolist()
print(top_bands[:10])

['402.8', '404.9', '762.1', '411', '764.1', '413.1', '400.8', '406.9', '760', '409']


In [191]:
def get_top_bands_comb(best_band,remain_bands, _input, label):
    clf = QuadraticDiscriminantAnalysis()
    cv = RepeatedKFold(n_splits=4,n_repeats=4, random_state=46)

    #best_band = [top_bands[0]]
    #remain_bands = top_bands[1:]

    print("Start to process bands combanation.....................")
    print("#######################################################")
    t0 = time()
    results = list()
    _best_score = 0.0

    for i in range(len(remain_bands)):
        _best_band = ' '
        result = dict()
        for item in remain_bands:
            selected_bands = best_band.copy()
            selected_bands.append(item)
            _new_input = _input[selected_bands]
            scores = cross_val_score(clf, _new_input, label, scoring='accuracy', cv=cv, n_jobs=-1)
            _score = np.mean(scores)

            if _score > _best_score:
                _best_score = _score
                _best_band = item

        if _best_band == ' ':  # Can't find more bands to increase score, then exit 
            break

        best_band.append(_best_band)
        remain_bands.remove(_best_band)
        result['score'] = _best_score
        result['bands'] = best_band.copy()

        results.append(result)

    print("Using {} sec".format(time()-t0))
    return results

In [192]:
best_band = [top_bands[0]]
remain_bands = top_bands[1:]
results = get_top_bands_comb(best_band, remain_bands, _input, label)

Start to process bands combanation.....................
#######################################################


KeyError: "None of [Index(['Chl green ', 'CI green'], dtype='object')] are in the [columns]"

In [71]:
print(results[-1]['bands'])

['402.8', '411', '764.1', '749.7', '774.4', '778.5', '745.6', '565', '552.7', '811.3', '801.1', '671.7', '870.9', '626.6']


In [19]:
bands_pd = pd.DataFrame(results)
bands_pd

,score,bands
0,0.729103,"[402.8, 411]"
1,0.754384,"[402.8, 411, 764.1]"
2,0.810527,"[402.8, 411, 764.1, 772.3]"
3,0.848783,"[402.8, 411, 764.1, 772.3, 737.4]"
4,0.874577,"[402.8, 411, 764.1, 772.3, 737.4, 706.6]"
5,0.892281,"[402.8, 411, 764.1, 772.3, 737.4, 706.6, 753.8]"
6,0.898013,"[402.8, 411, 764.1, 772.3, 737.4, 706.6, 753.8..."
7,0.901216,"[402.8, 411, 764.1, 772.3, 737.4, 706.6, 753.8..."


In [47]:
results[-1]['bands']

['402.8', '411', '764.1', '772.3', '737.4', '706.6', '753.8', '813.4', '690.2']

In [48]:
len(results[-1]['bands'])

9

In [ ]:
bands_pd.loc[15]['bands']

In [ ]:
len(bands_pd.loc[15]['bands'])

In [ ]:
clf = build_model()

In [ ]:
selected_bands =bands_pd.iloc[-1,:]['bands']

In [ ]:
X = _input[selected_bands].to_numpy()
y = label

rkf = RepeatedKFold(n_splits=4, n_repeats=4, random_state=2652124)
f1_scores = []
for train_index, test_index in rkf.split(X):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    _score = f1_score(y_test, y_pred, average=None)
    f1_scores.append(_score)

In [ ]:
f1_scores = np.asarray(f1_scores)
f1_scores

In [ ]:
np.mean(f1_scores, axis = 0), np.mean(f1_scores)

In [ ]:
np.std(f1_scores, axis = 0), np.std(f1_scores)

In [72]:
from itertools import combinations

In [ ]:
import operator as op
from functools import reduce

def ncr(n, r):
    r = min(r, n-r)
    numer = reduce(op.mul, range(n, n-r, -1), 1)
    denom = reduce(op.mul, range(1, r+1), 1)
    return numer // denom  # or / in Python 2

In [ ]:
num = ncr(20,6)
num

In [ ]:
clf = QuadraticDiscriminantAnalysis()
cv = RepeatedKFold(n_splits=4,n_repeats=4, random_state=46)

In [ ]:
results = list()
_best_score = 0.0
_best_band = set()

comb = combinations(selected_bands, 6)
count = 0
for item in comb:
    count += 1
    if count == 50:
        break
    _bands = set(item) # item is tuple, convert to set
    result = dict()
    #print(bands_set)
    _new_input = _input[_bands]
    scores = cross_val_score(clf, _new_input, label, scoring='accuracy', cv=cv, n_jobs=-1)
    _score = np.mean(scores)
    if _score > _best_score:
        _best_score = _score
        _best_band = _bands
        print(_best_score, _best_band)

In [ ]:
_best_score

In [ ]:
_best_band

## Univariate feature selection

In [57]:
# chi2
clf = SelectKBest(chi2, k=2)
X_new = clf.fit_transform(_input, label)
X_new.shape

(1483, 2)

In [58]:
clf.get_feature_names_out()

array(['762.1', '817.5'], dtype=object)

In [42]:
indices = np.argsort(clf.scores_)[::-1]
indices

array([179, 206, 207, 180, 208, 205, 204, 211, 203, 209, 210, 224, 202,
       194, 223, 201, 193, 213, 214, 200, 225, 226, 189, 195, 181, 188,
       196, 215, 192, 199, 233, 227, 222, 197, 190, 191, 198, 239, 221,
       238, 232, 237, 216, 235, 236, 220, 229, 234, 212, 178, 228, 187,
       230, 218, 231, 219, 217, 177, 176, 182, 175, 186, 183, 184, 185,
       174, 173, 172, 171, 170, 169, 168, 141, 140, 139, 138, 131, 142,
       137, 130, 134, 132, 133, 136, 135, 127, 129, 148, 146, 145, 128,
       143, 147, 120, 126, 167, 119, 121, 144, 122, 118, 123, 149, 125,
       117, 124, 116, 115, 114, 111, 112, 113, 110, 109, 108, 104, 105,
       103, 107, 150, 106, 102,   8, 166, 101, 165, 100, 151,  99,  98,
        93,  94,   9,  51,  92, 152,  52,  56,  57,  97,  50,  95,  49,
        43, 164,  55,  58,  42,  44,  53,  54,  96,  91,  48,  45,  59,
        41,  90,  46,  47,   7,  40,  35,  34,  89,  33, 153,  60,  63,
        36,  88,  39,  64,  37,  62,  29,  28,  26,  38,  27,  6

In [61]:
k=2
top_bands = list(_input.columns.values[indices])
len(top_bands)
#print(top_bands[:10])

240

In [62]:
results = get_top_bands_comb(top_bands, _input, label)

Start to process bands combanation.....................
#######################################################
Using 113.2526307106018 sec


In [86]:
print(results[-1]['bands'])

['762.1', '747.7', '753.8', '788.7', '655.3', '879.1', '774.4', '540.3', '823.6', '565', '597.8', '394.6']


In [80]:
clf = SelectKBest(f_classif, k=2)
X_new = clf.fit_transform(_input, label)
X_new.shape

(1483, 2)

In [81]:
indices = np.argsort(clf.scores_)[::-1]
indices

array([179,   8, 180, 206,   9, 207, 208, 205, 204, 211, 203, 181, 209,
       194, 224, 210, 178, 193, 223, 202, 189, 201, 225, 213, 200, 188,
       195, 214, 226, 192, 196, 190, 191, 215, 177, 227, 233, 239, 197,
       222, 199, 198, 176, 221, 238, 237, 235, 232, 216, 236, 220, 229,
       187, 234, 228, 140, 141, 212, 139, 138, 230, 218, 231, 131, 219,
       217, 130, 175, 142, 137, 132, 134, 133, 182, 127, 136, 135, 129,
       128,   7, 120, 119, 121, 126, 148, 146, 145, 143, 122, 186, 118,
       147,  13, 123, 144, 183, 117, 174, 125, 116,  43,  51, 149, 184,
       124,  10, 115,  42, 111, 112, 114, 185, 110, 113,  50,  52,  44,
        11,  49,  14, 173,  12,  26, 109,  34,  56, 104,  25,  27, 172,
       108,  35, 103,  57,  33, 105,  28,  41, 107,  29,  45, 106,  55,
        53,  48, 102,  54,  36,  19, 171,  15,  40, 101,  58,  16,  18,
        46,  20,  47,  37,  39, 100,  32,  22,  30, 170,  21,  24, 150,
        99,  38,  59,  17,  23,  31,  98,  93, 169,  94,  92,  9

In [83]:
#k=60
top_bands = list(_input.columns.values[indices])
print(top_bands)

['762.1', '411', '764.1', '817.5', '413.1', '819.5', '821.6', '815.4', '813.4', '827.7', '811.3', '766.2', '823.6', '792.8', '854.4', '825.7', '760', '790.8', '852.4', '809.3', '782.6', '807.2', '856.5', '831.8', '805.2', '780.5', '794.9', '833.9', '858.5', '788.7', '796.9', '784.6', '786.7', '836', '757.9', '860.6', '872.9', '885.2', '799', '850.3', '803.1', '801.1', '755.9', '848.3', '883.2', '881.1', '877', '870.9', '838', '879.1', '846.2', '864.7', '778.5', '875', '862.6', '682', '684', '829.8', '679.9', '677.9', '866.7', '842.1', '868.8', '663.5', '844.2', '840.1', '661.5', '753.8', '686.1', '675.8', '665.6', '669.7', '667.6', '768.2', '655.3', '673.8', '671.7', '659.4', '657.4', '409', '640.9', '638.9', '643', '653.3', '698.4', '694.3', '692.3', '688.1', '645', '776.4', '636.8', '696.4', '421.3', '647.1', '690.2', '770.3', '634.8', '751.8', '651.2', '632.7', '482.9', '499.3', '700.5', '772.3', '649.1', '415.1', '630.7', '480.8', '622.5', '624.5', '628.6', '774.4', '620.4', '626.6

In [84]:
results = get_top_bands_comb(top_bands, _input, label)

Start to process bands combanation.....................
#######################################################
Using 164.68579983711243 sec


In [89]:
print(results[-1]['bands'])

['762.1', '747.7', '753.8', '788.7', '655.3', '879.1', '774.4', '540.3', '823.6', '565', '597.8', '394.6']


### RFECV

In [171]:
from sklearn.model_selection import StratifiedKFold

In [236]:
svc = SVC(kernel="linear")
min_features_to_select = 6
rfecv = RFECV(
    estimator=svc,
    step=1,
    cv=StratifiedKFold(4),
    scoring="accuracy",
    min_features_to_select=min_features_to_select,
)
rfecv.fit(_input, label)

E:\anaconda3\envs\hsi\lib\site-packages\sklearn\base.py:446: UserWarning: X does not have valid feature names, but RFECV was fitted with feature names
  "X does not have valid feature names, but"


RFECV(cv=StratifiedKFold(n_splits=4, random_state=None, shuffle=False),
      estimator=SVC(kernel='linear'), min_features_to_select=6,
      scoring='accuracy')

In [237]:
rfecv.ranking_

array([28,  1, 16,  1,  1,  1,  1,  1,  1,  1,  1,  9,  1,  1,  1,  1,  1,
        6,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  7,  4, 10,  1,  1,
        1,  1, 12, 22, 23,  1,  1, 21, 29, 19, 20, 30, 25, 13,  2,  1,  1,
        1,  1,  1,  1,  1,  8,  1,  1,  1, 11, 31,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1, 27,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  3,  1,  1,  1,  1,
       14,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1, 32,  1,  1,  1,  1, 15, 26, 17,  1,
        1,  1,  1,  1,  1

In [240]:
_mask = np.ma.masked_where(rfecv.ranking_ == 1, rfecv.ranking_)
selected_bands = _input.columns.values[_mask.mask]
selected_bands.shape

(208,)

In [242]:
best_band = ['747.7', '762.1']
remain_bands = cols.copy()
remain_bands.remove('762.1')
remain_bands.remove('747.7')
results = get_top_bands_comb(best_band, remain_bands, _input, label)
results[-1]

Start to process bands combanation.....................
#######################################################
Using 101.1007628440857 sec


{'score': 0.9040854702411306,
 'bands': ['747.7',
  '762.1',
  '753.8',
  '788.7',
  '655.3',
  '879.1',
  '774.4',
  '540.3',
  '823.6',
  '565',
  '597.8',
  '394.6']}

In [175]:
from sklearn.ensemble import RandomForestClassifier

In [176]:
clf = RandomForestClassifier(random_state=46)
min_features_to_select = 6
rfecv = RFECV(
    estimator=clf,
    step=1,
    cv=StratifiedKFold(4),
    scoring="accuracy",
    min_features_to_select=min_features_to_select,
)
rfecv.fit(_input, label)

E:\anaconda3\envs\hsi\lib\site-packages\sklearn\base.py:446: UserWarning: X does not have valid feature names, but RFECV was fitted with feature names
  "X does not have valid feature names, but"


RFECV(cv=StratifiedKFold(n_splits=4, random_state=None, shuffle=False),
      estimator=RandomForestClassifier(random_state=46),
      min_features_to_select=6, scoring='accuracy')

In [177]:
rfecv.ranking_

array([  1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
         1,   1,  30,  15,  21,  65,   1,   1,  10,  97,  27,  67,  34,
         1,   5, 118, 125,  90, 184,  58, 151, 148,  68, 170, 140,  84,
       132, 114, 162, 105,  42,  98, 157,  48, 144,  75, 154, 183, 155,
        95,   1,  45, 116,  55,  80, 187, 143,   1,  87, 138,  59,   7,
        41,  56,   1,  99,  82, 107,   1, 142,  29,  36, 127,  92,  66,
         9,  20, 139, 131, 145, 176,  89, 165, 119,  73, 124, 134,   6,
       163,  23,  43, 102, 133,  53,  86, 174, 149, 147, 175, 181, 160,
        78, 168, 153, 172, 169, 130, 191, 161,  60, 100, 150, 113,  26,
       190, 110, 111, 137,  93,   1,  70,  12,   1,  64, 101, 112,  22,
        31,   1,  77,  74,  54,   1,  33,  28,  18,  39,   1,   1,  38,
        52,   1,  83,  85,   1,   1,   1,   1,   2,   1,  57,  76,  37,
         3,  11, 109,  17, 108,   1,  47,  14,  24,   4,   1,   1,  71,
       128, 106,  63,  51,  40, 156, 120,   1,   1,   1,   1,   

In [181]:
_mask = np.ma.masked_where(rfecv.ranking_ == 1, rfecv.ranking_)
_mask

masked_array(data=[--, --, --, --, --, --, --, --, --, --, --, --, --, --,
                   --, 30, 15, 21, 65, --, --, 10, 97, 27, 67, 34, --, 5,
                   118, 125, 90, 184, 58, 151, 148, 68, 170, 140, 84, 132,
                   114, 162, 105, 42, 98, 157, 48, 144, 75, 154, 183, 155,
                   95, --, 45, 116, 55, 80, 187, 143, --, 87, 138, 59, 7,
                   41, 56, --, 99, 82, 107, --, 142, 29, 36, 127, 92, 66,
                   9, 20, 139, 131, 145, 176, 89, 165, 119, 73, 124, 134,
                   6, 163, 23, 43, 102, 133, 53, 86, 174, 149, 147, 175,
                   181, 160, 78, 168, 153, 172, 169, 130, 191, 161, 60,
                   100, 150, 113, 26, 190, 110, 111, 137, 93, --, 70, 12,
                   --, 64, 101, 112, 22, 31, --, 77, 74, 54, --, 33, 28,
                   18, 39, --, --, 38, 52, --, 83, 85, --, --, --, --, 2,
                   --, 57, 76, 37, 3, 11, 109, 17, 108, --, 47, 14, 24, 4,
                   --, --, 71, 128, 10

In [189]:

selected_bands = _input.columns.values[_mask.mask]
selected_bands

array(['394.6', '396.7', '398.7', '400.8', '402.8', '404.9', '406.9',
       '409', '411', '413.1', '415.1', '417.2', '419.2', '421.3', '423.3',
       '433.6', '435.7', '448', '503.4', '517.8', '532.1', '540.3', '645',
       '651.2', '663.5', '671.7', '682', '684', '690.2', '696.4', '698.4',
       '700.5', '702.5', '706.6', '725.1', '735.4', '737.4', '755.9',
       '757.9', '760', '762.1', '764.1', '766.2', '772.3', '774.4',
       '811.3', '817.5', '819.5', '821.6', '827.7'], dtype=object)

In [239]:
t2 = time()

clf = QuadraticDiscriminantAnalysis()
cv = RepeatedKFold(n_splits=4,n_repeats=4, random_state=46)

_best_score = 0.0
_best_band = set()

comb = combinations(selected_bands, 2)
count = 0
for item in comb:
    _bands = set(item) # item is tuple, convert to set
    #result = dict()
    #print(bands_set)
    _new_input = _input[_bands]
    scores = cross_val_score(clf, _new_input, label, scoring='accuracy', cv=cv, n_jobs=-1)
    _score = np.mean(scores)
    if _score > _best_score:
        _best_score = _score
        _best_band = _bands

    count += 1
    if count % 1000 == 0:
        print("Now is {} and using {} seconds".format(count, time()-t2))

print("Using {} sec to do combination".format(time()-t2))
print(_best_band)
print(_best_score)

Now is 1000 and using 40.543601751327515 seconds
Now is 2000 and using 78.5659704208374 seconds
Now is 3000 and using 118.34248280525208 seconds
Now is 4000 and using 157.10329580307007 seconds
Now is 5000 and using 197.92251014709473 seconds
Now is 6000 and using 236.92824983596802 seconds
Now is 7000 and using 277.50479078292847 seconds
Now is 8000 and using 316.76883912086487 seconds
Now is 9000 and using 355.3656716346741 seconds
Now is 10000 and using 393.32819962501526 seconds
Now is 11000 and using 434.4811990261078 seconds
Now is 12000 and using 473.8469753265381 seconds
Now is 13000 and using 514.4673991203308 seconds
Now is 14000 and using 552.6433572769165 seconds
Now is 15000 and using 589.8399314880371 seconds
Now is 16000 and using 626.6924264431 seconds
Now is 17000 and using 663.2746429443359 seconds
Now is 18000 and using 701.9103715419769 seconds
Now is 19000 and using 740.3566064834595 seconds
Now is 20000 and using 778.7350234985352 seconds
Now is 21000 and using 81

In [194]:
best_band = ['762.1', '737.4']
remain_bands = cols.copy()
remain_bands.remove('762.1')
remain_bands.remove('737.4')
results = get_top_bands_comb(best_band, remain_bands, _input, label)

Start to process bands combanation.....................
#######################################################
Using 47.15196466445923 sec


In [195]:
results[-1]

{'score': 0.8819989801121877,
 'bands': ['762.1', '737.4', '721', '770.3', '753.8', '396.7']}

### Lasso

In [225]:
from sklearn.linear_model import RidgeClassifier
from sklearn import linear_model

In [197]:
parameters = { 'alpha':[1, 0.1, 0.01, 0.001, 0.0001]}
rcf = RidgeClassifier()
clf = GridSearchCV(rcf, parameters, cv=10)
clf.fit(_input, label)
clf.best_estimator_

RidgeClassifier(alpha=0.0001)

In [226]:
parameters = { 'alpha':[1, 0.1, 0.01, 0.001, 0.0001]}
lasso = linear_model.Lasso()
clf = GridSearchCV(lasso, parameters, cv=10)
clf.fit(_input, label)
clf.best_estimator_

E:\anaconda3\envs\hsi\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.318e+00, tolerance: 3.232e-02
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive
E:\anaconda3\envs\hsi\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.163e+00, tolerance: 3.232e-02
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive
E:\anaconda3\envs\hsi\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regular

Lasso(alpha=0.0001)

In [227]:
lasso = linear_model.Lasso(alpha=0.0001).fit(_input, label)
lasso.coef_

E:\anaconda3\envs\hsi\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.493e+00, tolerance: 3.695e-02
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive


array([  0.        ,   0.        ,   0.        ,  -0.        ,
        -1.9305835 ,  -0.        ,   0.        ,   0.        ,
         2.06472234,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.  

In [232]:
indices = np.argsort(lasso.coef_)[::-1]
top_bands = _input.columns.values[indices]
top_bands[:10]

array(['772.3', '735.4', '737.4', '770.3', '411', '774.4', '556.8',
       '548.6', '550.6', '552.7'], dtype=object)

In [233]:
best_band = ['772.3']
remain_bands = cols.copy()
remain_bands.remove('772.3')
results = get_top_bands_comb(best_band, remain_bands, _input, label)
results[-1]

Start to process bands combanation.....................
#######################################################
Using 57.11829400062561 sec


{'score': 0.9045872186202375,
 'bands': ['772.3', '792.8', '757.9', '766.2', '875', '823.6']}

In [1]:
rcf = RidgeClassifier(alpha=1).fit(_input, label)

NameError: name 'RidgeClassifier' is not defined

In [219]:
indices = np.argsort(rcf.coef_)[::-1]
top_bands = _input.columns.values[indices][0]
top_bands[:10]

array(['562.9', '710.7', '778.5', '805.2', '532.1', '827.7', '567',
       '649.1', '743.6', '801.1'], dtype=object)

In [220]:
best_band = ['562.9']
remain_bands = cols.copy()
remain_bands.remove('562.9')
results = get_top_bands_comb(best_band, remain_bands, _input, label)

Start to process bands combanation.....................
#######################################################
Using 94.1483166217804 sec


In [223]:
print(results[-1]['bands'])

['562.9', '762.1', '747.7', '768.2', '776.4', '538.3', '827.7', '872.9', '690.2', '575.2']


### R_regression

In [243]:
from sklearn.feature_selection import r_regression

In [244]:
_corr = r_regression(_input, label)

In [247]:
_corr

array([ 0.09440759,  0.12867053,  0.11733782,  0.04997425, -0.05903167,
       -0.00771339,  0.11583198,  0.20277391,  0.27120537,  0.24283762,
        0.18747352,  0.18367728,  0.1825396 ,  0.1960098 ,  0.18315604,
        0.1724648 ,  0.17148123,  0.16181763,  0.17015251,  0.17280389,
        0.16967571,  0.16724355,  0.16773493,  0.16090204,  0.16713735,
        0.17937236,  0.18225478,  0.17919408,  0.17822999,  0.17706177,
        0.16756474,  0.16027438,  0.16807373,  0.17844175,  0.18023552,
        0.17875347,  0.17327414,  0.16857848,  0.16274609,  0.16826941,
        0.17191681,  0.17760222,  0.18674359,  0.18824558,  0.18408333,
        0.17666896,  0.17000303,  0.16868014,  0.17492899,  0.18317473,
        0.18518291,  0.18792145,  0.18415016,  0.17547044,  0.17393167,
        0.17558801,  0.17977394,  0.17847786,  0.17159356,  0.16242187,
        0.14920136,  0.1397673 ,  0.1417104 ,  0.14311398,  0.13928734,
        0.12957646,  0.12403603,  0.11702498,  0.11016569,  0.10

In [248]:
indices = np.argsort(_corr)[::-1]
indices

array([  8,   9, 140, 141, 139, 138, 131, 130, 142, 137, 132, 134, 133,
       127, 136, 135, 129, 128,   7, 120, 119, 121, 126, 148, 146, 145,
       143, 122, 118, 147,  13, 123, 144, 117, 125, 116,  43,  51, 149,
       124,  10, 115,  42, 111, 112, 114, 110, 113,  50,  52,  44,  11,
        49,  14,  12,  26, 109,  34,  56, 104,  25,  27, 108,  35, 103,
        57,  33, 105,  28,  41, 107,  29,  45, 106,  55,  53,  48, 102,
        54,  36,  19,  15,  40, 101,  58,  16,  18,  46,  20,  47,  37,
        39, 100,  32,  22,  30,  21,  24, 150,  99,  38,  59,  17,  23,
        31,  98,  93,  94,  92,  97, 151,  60,  95,  96,  63,  91,  62,
        61,  64, 152,  90,  89,  65,   1,  88,  66,  87,   2,  67,   6,
       153,  86,  85,  68,  84,  69,  83,  70,  79,  71,  78,  82,  74,
       154,   0,  75,  76,  77,  80,  73,  72,  81, 155, 156, 157,   3,
         5, 158, 161, 160, 159, 162,   4, 163, 164, 166, 165, 167, 168,
       169, 170, 171, 172, 173, 185, 184, 174, 183, 186, 182, 17

In [252]:
top_bands = list(_input.columns.values[indices])
print(top_bands[:10])

['411', '413.1', '682', '684', '679.9', '677.9', '663.5', '661.5', '686.1', '675.8']


In [250]:
best_band = ['411']
remain_bands = cols.copy()
remain_bands.remove('411')
results = get_top_bands_comb(best_band, remain_bands, _input, label)

Start to process bands combanation.....................
#######################################################
Using 173.30756521224976 sec


In [254]:
print(results[-1]['bands'])

['411', '404.9', '762.1', '749.7', '774.4', '778.5', '731.3', '696.4', '817.5', '885.2', '803.1', '675.8']


## SFS

In [255]:
from sklearn.feature_selection import SequentialFeatureSelector

In [256]:
clf = QuadraticDiscriminantAnalysis()

In [258]:
sfs = SequentialFeatureSelector(clf, n_features_to_select=3, cv=6)
sfs.fit(_input, label)

SequentialFeatureSelector(cv=6, estimator=QuadraticDiscriminantAnalysis(),
                          n_features_to_select=3)

In [263]:
sfs.get_support()

array([False, False, False, False, False,  True, False, False,  True,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [260]:
sfs.transform(_input).shape

(1483, 3)

In [264]:
_input.columns.values[sfs.get_support()]

array(['404.9', '411', '762.1'], dtype=object)

In [265]:
best_band = ['404.9']
remain_bands = cols.copy()
remain_bands.remove('404.9')
results = get_top_bands_comb(best_band, remain_bands, _input, label)

Start to process bands combanation.....................
#######################################################
Using 111.87994909286499 sec


In [268]:
print(results[-1]['bands'])

['404.9', '411', '762.1', '749.7', '774.4', '778.5', '731.3', '696.4', '817.5', '885.2', '803.1', '675.8']


## Use Combination

In [73]:
all_bands = _input.columns.values

In [74]:
print("Start to find best 2 bands via combination method ................")

t2 = time()

clf = QuadraticDiscriminantAnalysis()
cv = RepeatedKFold(n_splits=4,n_repeats=4, random_state=46)

_best_score = 0.0
_best_band = set()

comb = combinations(all_bands, 2)
for item in comb:
    _bands = set(item) # item is tuple, convert to set
    result = dict()
    #print(bands_set)
    _new_input = _input[_bands]
    scores = cross_val_score(clf, _new_input, label, scoring='accuracy', cv=cv, n_jobs=-1)
    _score = np.mean(scores)
    if _score > _best_score:
        _best_score = _score
        _best_band = _bands

print("Using {} sec to do combination".format(time()-t2))
print(_best_band)
print(_best_score)

Start to find best 2 bands via combination method ................
Using 1059.0038137435913 sec to do combination
{'747.7', '762.1'}
0.8417125009106141


In [76]:
clf = QuadraticDiscriminantAnalysis()
cv = RepeatedKFold(n_splits=4,n_repeats=4, random_state=46)

best_band = ['747.7', '762.1']
remain_bands = all_bands.tolist().copy()
remain_bands.remove('747.7')
remain_bands.remove('762.1')

print("Start to process bands selection.....................")
print("#######################################################")
t0 = time()
results = list()
_best_score = 0.0

for i in range(len(remain_bands)):
    _best_band = ' '
    result = dict()
    for item in remain_bands:
        selected_bands = best_band.copy()
        selected_bands.append(item)
        _new_input = _input[selected_bands]
        scores = cross_val_score(clf, _new_input, label, scoring='accuracy', cv=cv, n_jobs=-1)
        _score = np.mean(scores)

        if _score > _best_score:
            _best_score = _score
            _best_band = item

    if _best_band == ' ':  # Can't find more bands to increase score, then exit 
        break

    best_band.append(_best_band)
    remain_bands.remove(_best_band)
    result['score'] = _best_score
    result['bands'] = best_band.copy()

    results.append(result)

Start to process bands selection.....................
#######################################################


In [79]:
print(results[-1]['bands'])

['747.7', '762.1', '753.8', '788.7', '655.3', '879.1', '774.4', '540.3', '823.6', '565', '597.8', '394.6']


In [5]:
tree_short = set(['402.8', '411', '764.1', '772.3', '737.4', '706.6', '753.8', '813.4', '690.2'])
tree_lists = set(['402.8', '411', '764.1', '749.7', '774.4', '778.5', '745.6', '565', '552.7', '811.3', '801.1', '671.7', '870.9', '626.6'])
ch2_lists = set(['762.1', '747.7', '753.8', '788.7', '655.3', '879.1', '774.4', '540.3', '823.6', '565', '597.8', '394.6'])
comb_lists = set(['747.7', '762.1', '753.8', '788.7', '655.3', '879.1', '774.4', '540.3', '823.6', '565', '597.8', '394.6'])
rfe_randomforest_lists = set(['762.1', '737.4', '721', '770.3', '753.8', '396.7'])
rfe_svc_lists = set(['747.7', '762.1', '753.8', '788.7', '655.3', '879.1', '774.4', '540.3', '823.6', '565', '597.8', '394.6'])
corr_pr_lists = set(['411', '404.9', '762.1', '749.7', '774.4', '778.5', '731.3', '696.4', '817.5', '885.2', '803.1', '675.8'])
sfs_forward_lists = set(['404.9', '411', '762.1', '749.7', '774.4', '778.5', '731.3', '696.4', '817.5', '885.2', '803.1', '675.8'])
ridge_lists = set(['562.9', '762.1', '747.7', '768.2', '776.4', '538.3', '827.7', '872.9', '690.2', '575.2'])
lasso_lists = set(['772.3', '792.8', '757.9', '766.2', '875', '823.6'])

full_lists = set.union(tree_short,tree_lists,ch2_lists,comb_lists, 
                rfe_randomforest_lists, rfe_svc_lists, corr_pr_lists, 
                sfs_forward_lists, ridge_lists, lasso_lists)

In [6]:
len(full_lists)

50

In [7]:
full_lists = list(full_lists)

In [286]:
_len = len(full_lists)
_index_list = [i for i in range(_len)]

comb = combinations(_index_list, 6)
count = 0
for item in comb:
    _bands = [full_lists[idx] for idx in item] #set(item) # item is tuple, convert to set
    break

In [288]:
_bands, item

(['696.4', '801.1', '811.3', '768.2', '885.2', '870.9'], (0, 1, 2, 3, 4, 5))

In [289]:
_new = _input[_bands]

In [290]:
_new

,696.4,801.1,811.3,768.2,885.2,870.9
0,0.05203,0.2992,0.3029,0.2793,0.3194,0.3175
1,0.05237,0.3195,0.3238,0.2975,0.3416,0.3393
2,0.05096,0.3437,0.3476,0.3209,0.3648,0.3638
3,0.05404,0.3213,0.3255,0.2995,0.3432,0.3390
4,0.05635,0.2212,0.2275,0.2047,0.2471,0.2431
...,...,...,...,...,...,...
1478,0.06635,0.1798,0.1809,0.1698,0.1964,0.1907
1479,0.07391,0.2053,0.2078,0.1934,0.2168,0.2208
1480,0.06157,0.2002,0.2020,0.1903,0.2161,0.2150
1481,0.07138,0.2581,0.2592,0.2457,0.2735,0.2725


In [271]:
from datetime import datetime

In [272]:
now = datetime.now()
 
print("now =", now)

now = 2021-12-29 09:51:43.625080


In [2]:
import csv

In [278]:
index_list = [i for i in range(50)]
index_comb = combinations(index_list, 6)
count = 0
with open("comb_6_test.csv", 'w', newline='') as f:
    f_writer = csv.writer(f, delimiter=',')
    for item in index_comb:
        f_writer.writerow(item)
        count +=1

print(count)

15890700


In [10]:
with open("comb_6_test.csv", newline='') as f:
    f_reader = csv.reader(f, delimiter=',')
    for row in f_reader:
        print(row)
        print([int(i) for i in row])
        print([full_lists[int(i)] for i in row])
        break

['0', '1', '2', '3', '4', '5']
[0, 1, 2, 3, 4, 5]
['552.7', '801.1', '575.2', '690.2', '675.8', '776.4']


### Select best six bands from 24

In [153]:
# Use combination method to try all six bands combations
output = './output/'
bands_file = output + 'tree_chi_comb_6_bands.pkl'
_bands = pickle.load(open(bands_file, 'rb'))
_bands

{'706.6', '737.4', '764.1', '774.4', '801.1', '813.4'}

#### Use those bands to test other stage 

In [159]:
def process_data(file_name, sheet_name, selected_bands):
    full_data = pd.read_excel(file_name, sheet_name= sheet_name)

    str_cols = full_data.columns.astype(str)
    full_data.columns = str_cols
    cols = str_cols.tolist()
    cols = cols[1:]

    _input = full_data[cols]
    label = full_data['label'].tolist()
    label = np.asarray(label).reshape(-1,)

    clf = QuadraticDiscriminantAnalysis()
    
    X = _input[selected_bands].to_numpy()
    y = label

    print("Start to cal best_bands f1 score .....................")
    print("#######################################################")
    t1 = time()
    rkf = RepeatedKFold(n_splits=4, n_repeats=4, random_state=46)
    f1_scores = []
    for train_index, test_index in rkf.split(X):
        #print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        clf.fit(X_train,y_train)
        y_pred = clf.predict(X_test)
        _score = f1_score(y_test, y_pred, average=None)
        f1_scores.append(_score)

    f1_scores = np.asarray(f1_scores)
    print(sheet_name)
    print(np.mean(f1_scores, axis=0))

In [160]:
file_name = 'Adjusted categorization - 1-5,16-30, 35-70.xlsx'
sheet_list = ['H vs. Sc 16-34', 'H vs. Sc35-70','Multiclasses']

selected_bands = ['706.6', '737.4', '764.1', '774.4', '801.1', '813.4']

for sheet_name in sheet_list:
    process_data(file_name, sheet_name, selected_bands)

Start to cal best_bands f1 score .....................
#######################################################
H vs. Sc 16-34
[0.93001856 0.96209848]
Start to cal best_bands f1 score .....................
#######################################################
H vs. Sc35-70
[0.95338444 0.9351629 ]
Start to cal best_bands f1 score .....................
#######################################################
Multiclasses
[0.85302569 0.52955449 0.66397869 0.70371625]


### Results of selected bands

#### All classes

In [19]:
output = './output/'
bands = output + 'All classesselected_bands.pkl'
f1_scores = output + 'All classesselected_bands_f1_scores.pkl'

In [20]:
_bands = pickle.load(open(bands,'rb'))
print(_bands)

['402.8', '679.9', '411', '762.1', '725.1', '606', '710.7', '708.7', '694.3', '819.5', '513.7', '458.2', '653.3']


In [7]:
_f1_score = pickle.load(open(f1_scores, 'rb'))
_f1_score

array([[0.75886525, 0.70943396, 0.68862275, 0.81666667],
       [0.75      , 0.78354978, 0.71794872, 0.81422925],
       [0.80588235, 0.78787879, 0.6295082 , 0.79487179],
       [0.7826087 , 0.75449102, 0.73684211, 0.85530547],
       [0.80526316, 0.7943662 , 0.71299094, 0.85      ],
       [0.82242991, 0.77384196, 0.74056604, 0.91240876],
       [0.86863271, 0.80965147, 0.77368421, 0.88461538],
       [0.77530864, 0.72835821, 0.75824176, 0.82978723],
       [0.81818182, 0.78240741, 0.71153846, 0.86538462],
       [0.83838384, 0.78309859, 0.77435897, 0.84897959],
       [0.79084967, 0.77245509, 0.72384937, 0.84328358],
       [0.75862069, 0.80275229, 0.67936508, 0.7942029 ],
       [0.80093677, 0.79347826, 0.7266881 , 0.84285714],
       [0.85245902, 0.74172185, 0.72888889, 0.88059701],
       [0.83495146, 0.79237288, 0.70948012, 0.85611511],
       [0.79635258, 0.79665738, 0.65420561, 0.74801061]])

In [8]:
np.mean(_f1_score, axis=0)

array([0.80373291, 0.7754072 , 0.71667371, 0.83983219])

In [9]:
bands = output + 'All classes6_bands.pkl'
f1_scores = output + 'All classes6_bands_f1_scores.pkl'

In [10]:
_bands = pickle.load(open(bands,'rb'))
print(_bands)

{'458.2', '513.7', '725.1', '762.1', '606', '710.7'}


In [11]:
_f1_score = pickle.load(open(f1_scores, 'rb'))
np.mean(_f1_score, axis=0)

array([0.75306985, 0.71921036, 0.68881343, 0.83388149])

In [6]:
_bands = ['737.4', '811.3', '762.1', '706.6', '774.4', '803.1']

### To calculate each classifcation method

In [7]:
def get_f1_score_by_method(clf, _input, label, selected_bands):
    X = _input[selected_bands].to_numpy()
    y = label
    rkf = RepeatedKFold(n_splits=4, n_repeats=4, random_state=2652124)
    f1_scores = []
    for train_index, test_index in rkf.split(X):
        #print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        clf.fit(X_train,y_train)
        y_pred = clf.predict(X_test)
        _score = f1_score(y_test, y_pred, average=None)
        f1_scores.append(_score)
        
    return f1_scores

In [9]:
# #############################################################################
# Train a SVM classification model

print("Fitting the classifier to the training set")
t0 = time()
param_grid = {'kernel':['rbf','sigmod','linear'],'C': [1e2,5e2,1e3, 5e3, 1e4],
              'gamma': [0.001, 0.005, 0.01,0.05,0.1,0.2]}
print("Set Grid parameters")
clf = GridSearchCV(
    SVC(class_weight='balanced',probability=True), param_grid, cv=10)
print("Start to fit")
clf = clf.fit(_input[_bands], label)
print("done in %0.3fs" % (time() - t0))
print("Best estimator found by grid search:")
print(clf.best_estimator_)

Fitting the classifier to the training set
Set Grid parameters
Start to fit


E:\anaconda3\envs\hsi\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
300 fits failed out of a total of 900.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
300 fits failed with the following error:
Traceback (most recent call last):
  File "E:\anaconda3\envs\hsi\lib\site-packages\sklearn\model_selection\_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "E:\anaconda3\envs\hsi\lib\site-packages\sklearn\svm\_base.py", line 255, in fit
    fit(X, y, sample_weight, solver_type, kernel, random_seed=seed)
  File "E:\anaconda3\envs\hsi\lib\site-packages\sklearn\svm\_base.py", line 333, in _dense_fit
    random_seed=random_seed,
  File "sklearn\svm\_libsvm.pyx",

done in 1214.128s
Best estimator found by grid search:
SVC(C=10000.0, class_weight='balanced', gamma=0.001, kernel='linear',
    probability=True)


In [8]:
clf = SVC(C=10000.0, class_weight='balanced', gamma=0.001, kernel='linear',probability=True)

f1_scores = get_f1_score_by_method(clf, _input, label, _bands)
np.mean(f1_scores, axis=0)

array([0.82409481, 0.37977875, 0.58100207, 0.71703733])

In [24]:
## Decision Tree 
print("Fitting the classifier to the training set")
t0 = time()
param_grid = {'max_depth': range(1,20), 'criterion':['gini','entropy']}
print("Set Grid parameters")
clf = GridSearchCV(
    DecisionTreeClassifier(), param_grid, cv=10)
print("Start to fit")
clf = clf.fit(_input[_bands], label)
print("done in %0.3fs" % (time() - t0))
print("Best estimator found by grid search:")
print(clf.best_estimator_)

Fitting the classifier to the training set
Set Grid parameters
Start to fit
done in 11.501s
Best estimator found by grid search:
DecisionTreeClassifier(criterion='entropy', max_depth=17)


In [10]:
clf = DecisionTreeClassifier(criterion='entropy', max_depth=17)
f1_scores = get_f1_score_by_method(clf, _input, label, _bands)
np.mean(f1_scores, axis=0)

array([0.74481711, 0.69486073, 0.94728777, 0.78802054])

In [33]:
## Random forest

print("Fitting the classifier to the training set")
t0 = time()
param_grid = {'n_estimators':[50,100,150],'max_depth': range(1,20), 'criterion':['gini','entropy']}
print("Set Grid parameters")
clf = GridSearchCV(
    RandomForestClassifier(), param_grid, cv=10)
print("Start to fit")
clf = clf.fit(_input[_bands], label)
print("done in %0.3fs" % (time() - t0))
print("Best estimator found by grid search:")
print(clf.best_estimator_)

Fitting the classifier to the training set
Set Grid parameters
Start to fit
done in 661.924s
Best estimator found by grid search:
RandomForestClassifier(max_depth=19, n_estimators=150)


In [11]:
clf = RandomForestClassifier(max_depth=19, n_estimators=150)
f1_scores = get_f1_score_by_method(clf, _input, label, _bands)
np.mean(f1_scores, axis=0)

array([0.81079512, 0.76291904, 0.97070069, 0.84069884])

In [39]:
## Gaussian process classification (GPC)
kernel = 1.0 * RBF(1.0)
clf = GaussianProcessClassifier(kernel=kernel, random_state=46)
f1_scores = get_f1_score_by_method(clf, _input, label, _bands)
np.mean(f1_scores, axis=0)

E:\anaconda3\envs\hsi\lib\site-packages\sklearn\gaussian_process\kernels.py:437: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  ConvergenceWarning,
E:\anaconda3\envs\hsi\lib\site-packages\sklearn\gaussian_process\kernels.py:437: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  ConvergenceWarning,
E:\anaconda3\envs\hsi\lib\site-packages\sklearn\gaussian_process\kernels.py:437: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  ConvergenceWarning,
E:\anaconda3\envs\hsi\lib\site-packages\sklearn\gaussian_process\_g

array([0.86089773, 0.7413385 , 0.71967071, 0.75621773])

In [12]:
## AdaBoostClassifier

clf = AdaBoostClassifier(n_estimators=100, random_state=46)
f1_scores = get_f1_score_by_method(clf, _input, label, _bands)
np.mean(f1_scores, axis=0)

array([0.59848993, 0.40558774, 0.62259169, 0.57321023])

In [13]:
### KNeighborsClassifier
clf = KNeighborsClassifier(n_neighbors=3)
f1_scores = get_f1_score_by_method(clf, _input, label, _bands)
np.mean(f1_scores, axis=0)

array([0.73852572, 0.65929367, 0.91988101, 0.74632863])

In [14]:
### Gaussian Naive Bayes (GaussianNB)
clf = GaussianNB()
f1_scores = get_f1_score_by_method(clf, _input, label, _bands)
np.mean(f1_scores, axis=0)

array([0.46461791, 0.04060836, 0.42435429, 0.52288325])

In [15]:
### Quadratic Discriminant Analysis
clf = QuadraticDiscriminantAnalysis()
f1_scores = get_f1_score_by_method(clf, _input, label, _bands)
print("QDA:", np.mean(f1_scores, axis=0))

QDA: [0.85912796 0.48288606 0.64879602 0.72349269]


In [57]:
def process_modes_bands(_input, label, _bands):
    qda = QuadraticDiscriminantAnalysis()
    gnb = GaussianNB()
    knc = KNeighborsClassifier(n_neighbors=3)
    abc = AdaBoostClassifier(n_estimators=100, random_state=46)
    #gpc = GaussianProcessClassifier(kernel=(1.0 * RBF(1.0)), random_state=46)
    rfc = RandomForestClassifier(max_depth=19, n_estimators=150)
    dtc = DecisionTreeClassifier(criterion='entropy', max_depth=17)
    svc = SVC(C=10000.0, class_weight='balanced', gamma=0.001, kernel='linear',probability=True)
    # "Gaussian process classification",
    model_name = ["SVC","DecisionTreeClassifier","RandomForestClassifier","AdaBoostClassifier","KNeighborsClassifier","Gaussian Naive Bayes","Quadratic Discriminant Analysis"]
    #model = [svc,dtc,rfc,gpc,abc,knc,gnb,qda]
    model = [svc,dtc,rfc,abc,knc,gnb,qda]
    
    for name, clf in zip(model_name, model):
        f1_scores = get_f1_score_by_method(clf, _input, label, _bands)
        print(name, np.mean(f1_scores, axis=0), np.mean(f1_scores))

In [58]:
process_modes_bands(_input, label, _bands)

SVC [0.83209549 0.39793824 0.59215063 0.71194415] 0.6335321279265141
DecisionTreeClassifier [0.74742072 0.69955222 0.94448609 0.79460801] 0.7965167619728803
RandomForestClassifier [0.81150999 0.76564679 0.97171395 0.83935878] 0.8470573799433302
AdaBoostClassifier [0.59782944 0.40536198 0.622479   0.57312736] 0.5496994424560316
KNeighborsClassifier [0.71752605 0.63895369 0.9133929  0.73739444] 0.7518167687980606
Gaussian Naive Bayes [0.46461791 0.04060836 0.42435429 0.52288325] 0.36311595329074803
Quadratic Discriminant Analysis [0.85912796 0.48288606 0.64879602 0.72349269] 0.6785756816897168


In [59]:
_bands

['737.4', '811.3', '762.1', '706.6', '774.4', '803.1']

#### SC 1-15

In [54]:
def read_data(file_name, sheet_name):
    full_data = pd.read_excel(file_name, sheet_name= sheet_name)

    str_cols = full_data.columns.astype(str)
    full_data.columns = str_cols
    cols = str_cols.tolist()
    cols = cols[1:]

    _input = full_data[cols]
    label = full_data['label'].tolist()
    label = np.asarray(label).reshape(-1,)
    
    return _input, label

In [55]:
_input, label = read_data('An-3 classis.xlsx', 'H vs. Sc 1-15')

In [56]:
bands = output + 'H vs. Sc 1-15selected_bands.pkl'
f1_scores = output + 'H vs. Sc 1-15selected_bands_f1_scores.pkl'

In [57]:
_bands = pickle.load(open(bands,'rb'))
print(_bands)

['402.8', '411', '764.1', '772.3', '737.4', '706.6', '753.8', '811.3', '796.9', '673.8']


In [14]:
_f1_score = pickle.load(open(f1_scores, 'rb'))
np.mean(_f1_score, axis=0)

array([0.89903147, 0.91276079])

In [15]:
bands = output + 'H vs. Sc 1-156_bands.pkl'
f1_scores = output + 'H vs. Sc 1-156_bands_f1_scores.pkl'

In [16]:
_bands = pickle.load(open(bands,'rb'))
print(_bands)

{'737.4', '772.3', '764.1', '811.3', '706.6', '411'}


In [17]:
_f1_score = pickle.load(open(f1_scores, 'rb'))
np.mean(_f1_score, axis=0)

array([0.88852156, 0.90178046])

In [58]:
process_modes_bands(_input, label, _bands)

SVC [0.88028741 0.88653552]
DecisionTreeClassifier [0.78187605 0.80921124]
RandomForestClassifier [0.83625664 0.86013436]


E:\anaconda3\envs\hsi\lib\site-packages\sklearn\gaussian_process\kernels.py:437: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  ConvergenceWarning,


Gaussian process classification [0.90995623 0.91904795]
AdaBoostClassifier [0.79251756 0.81745988]
KNeighborsClassifier [0.78588791 0.81567431]
Gaussian Naive Bayes [0.64063319 0.63631196]
Quadratic Discriminant Analysis [0.90016909 0.9022809 ]


#### SC 16-49

In [59]:
_input, label = read_data('An-3 classis.xlsx', 'H vs. Sc 16-49')

In [60]:
bands = output + 'H vs. Sc 16-49selected_bands.pkl'
f1_scores = output + 'H vs. Sc 16-49selected_bands_f1_scores.pkl'

In [61]:
_bands = pickle.load(open(bands,'rb'))
print(_bands)

['708.7', '762.1', '731.3', '879.1', '827.7', '406.9', '749.7', '794.9', '831.8', '513.7']


In [62]:
process_modes_bands(_input, label, _bands)

SVC [0.91461394 0.91330157]
DecisionTreeClassifier [0.81159458 0.81723326]
RandomForestClassifier [0.86548717 0.86740369]


E:\anaconda3\envs\hsi\lib\site-packages\sklearn\gaussian_process\_gpc.py:472: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)


Gaussian process classification [0.9312678  0.93156288]
AdaBoostClassifier [0.84862771 0.847201  ]
KNeighborsClassifier [0.83109108 0.83073499]
Gaussian Naive Bayes [0.6540953  0.64667713]
Quadratic Discriminant Analysis [0.94264184 0.94042401]


In [ ]:
_f1_score = pickle.load(open(f1_scores, 'rb'))
np.mean(_f1_score, axis=0)

In [ ]:
bands = output + 'H vs. Sc 16-496_bands.pkl'
f1_scores = output + 'H vs. Sc 16-496_bands_f1_scores.pkl'

In [ ]:
_bands = pickle.load(open(bands,'rb'))
print(_bands)

In [ ]:
_f1_score = pickle.load(open(f1_scores, 'rb'))
np.mean(_f1_score, axis=0)

#### SC 50-70

In [63]:
_input, label = read_data('An-3 classis.xlsx','H vs. 50-70')

In [64]:
bands = output + 'H vs. 50-70selected_bands.pkl'
f1_scores = output + 'H vs. 50-70selected_bands_f1_scores.pkl'

In [65]:
_bands = pickle.load(open(bands,'rb'))
print(_bands)

['692.3', '768.2', '706.6', '770.3', '671.7', '468.5', '694.3', '435.7']


In [66]:
process_modes_bands(_input, label, _bands)

SVC [0.93482319 0.92217835]
DecisionTreeClassifier [0.92942643 0.91875578]
RandomForestClassifier [0.9423268  0.93257181]


E:\anaconda3\envs\hsi\lib\site-packages\sklearn\gaussian_process\kernels.py:437: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  ConvergenceWarning,
E:\anaconda3\envs\hsi\lib\site-packages\sklearn\gaussian_process\kernels.py:437: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  ConvergenceWarning,
E:\anaconda3\envs\hsi\lib\site-packages\sklearn\gaussian_process\kernels.py:437: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  ConvergenceWarning,
E:\anaconda3\envs\hsi\lib\site-packages\sklearn\gaussian_process\ke

Gaussian process classification [0.96111035 0.95261082]
AdaBoostClassifier [0.92659921 0.91211907]
KNeighborsClassifier [0.90122686 0.88319416]
Gaussian Naive Bayes [0.82749324 0.75708855]
Quadratic Discriminant Analysis [0.9619865  0.95317987]


In [ ]:
_f1_score = pickle.load(open(f1_scores, 'rb'))
np.mean(_f1_score, axis=0)

In [ ]:
bands = output + 'H vs. 50-706_bands.pkl'
f1_scores = output + 'H vs. 50-706_bands_f1_scores.pkl'

In [ ]:
_bands = pickle.load(open(bands,'rb'))
print(_bands)

In [ ]:
_f1_score = pickle.load(open(f1_scores, 'rb'))
np.mean(_f1_score, axis=0)

# For Veg Index

In [90]:
file_name = 'Adjusted categorization - 1-5,16-30, 35-70.xlsx'
sheet_name = 'spss Veg class1'
#sheet_list = ['Multi classes', 'Veg H vs. Vge Class 1','Vge H vs. Vege Class 2','Vege H vs. Class 3']

In [91]:
def process_data(file_name, sheet_name):
    full_data = pd.read_excel(file_name, sheet_name= sheet_name)

    str_cols = full_data.columns.astype(str)
    full_data.columns = str_cols
    cols = str_cols.tolist()
    cols = cols[1:]

    _input = full_data[cols]
    label = full_data['label'].tolist()
    label = np.asarray(label).reshape(-1,)
    
    return _input, label

In [101]:
_input, label = process_data(file_name, sheet_name)
_input.head()

,RARSa,RARSb,RARSc,PSSRa,NDVI 780,NDVI 705,GNDVI,PRI,SR860,SIPI (Car/Chl),...,ARI,Chl green,CI green,CI rededge 710,CVI,BWDRVI,BRI,SIPI,OSAVI,VARI
0,0.610325,13.871020,8.258915,7.904888,0.767869,0.323288,0.605223,-0.060532,8.322325,1.050201,...,-2.185732,0.244561,3.435878,2.016180,2.526708,0.169664,-7.305255,7.208506,0.607903,0.343023
1,0.599015,13.500460,8.741801,8.495081,0.781127,0.333922,0.618131,-0.063535,8.949747,1.045301,...,-2.299157,0.234162,3.647273,2.122353,2.591657,0.201354,-7.196111,7.677220,0.629819,0.354222
2,0.586636,13.588985,9.401429,9.512870,0.802686,0.347431,0.636815,-0.059957,9.980626,1.036122,...,-2.777379,0.220493,3.929719,2.364507,2.602739,0.235886,-8.080823,8.236271,0.658623,0.385905
3,0.614918,13.184357,8.179303,8.144487,0.773471,0.320628,0.611467,-0.059628,8.527250,1.044031,...,-2.364785,0.241035,3.509474,2.102919,2.544925,0.154925,-7.360053,6.833180,0.625231,0.352928
4,0.721288,14.826070,5.858263,4.817073,0.646843,0.221469,0.549646,-0.063285,5.219948,1.122946,...,0.271181,0.292772,2.740277,1.377069,2.919997,0.021709,1.225954,5.111679,0.475080,0.154166


In [95]:
etc = ExtraTreesClassifier(n_estimators=600)
etc = etc.fit(_input, label)

indices = np.argsort(etc.feature_importances_)[::-1]
indices

array([24, 27, 10,  7, 18, 11,  6, 17, 25,  0,  2, 26, 30,  5, 20, 12, 28,
       21,  8,  4,  1, 32, 29, 15,  3, 23, 19, 31, 14, 22,  9, 13, 16],
      dtype=int64)

In [96]:
top_bands = list(_input.columns.values[indices])
top_bands[:10]

['Chl green ',
 'CVI',
 'NPQI',
 'PRI',
 'GNDVI.1',
 'NDVI 761',
 'GNDVI',
 'RVSI',
 'CI green',
 'RARSa']

In [97]:
results = get_top_bands_comb(top_bands, _input, label)

Start to process bands combanation.....................
#######################################################
Using 16.31860899925232 sec


In [99]:
results[-1]

{'score': 0.8589094485320901,
 'bands': ['Chl green ',
  'GNDVI.1',
  'NDVI 780',
  'RDVI',
  'TVI',
  'RVSI',
  'PRI',
  'SR860',
  'PSSRa',
  'MCAR 1',
  'RARSc',
  'SIPI',
  'VARI',
  'ARI']}

In [103]:
# f_classif
clf = SelectKBest(f_classif, k=7)
X_new = clf.fit_transform(_input, label)
X_new.shape

(1483, 7)

In [105]:
indices = np.argsort(clf.scores_)[::-1]
indices

array([24, 25,  6, 18, 27, 30,  2, 11, 28, 26, 20,  8,  3, 12,  4, 31, 15,
       10, 17, 14,  9, 13, 22, 16, 23,  5, 32, 29,  0,  1,  7, 21, 19],
      dtype=int64)

In [106]:
top_bands = list(_input.columns.values[indices])
top_bands[:10]

['Chl green ',
 'CI green',
 'GNDVI',
 'GNDVI.1',
 'CVI',
 'SIPI',
 'RARSc',
 'NDVI 761',
 'BWDRVI',
 'CI rededge 710']

### All best bands 

In [107]:
xl = pd.ExcelFile('Adjusted categorization - 1-5,16-30, 35-70.xlsx')
name_list = xl.sheet_names
name_list.remove('All categories')
name_list.remove('VEG index')

In [109]:
print(name_list)

['H vs. 1-15', 'H vs. Sc35-70', 'H vs. Sc 16-34', 'Multiclasses', 'spss Veg class1', 'Spss veg class 2', 'Spss veg class 3', 'SPSS multi VEG adjusted']


In [7]:
output = './output/'
#name_list = ['Multi classes', 'Veg H vs. Vge Class 1','Vge H vs. Vege Class 2','Vege H vs. Class 3']
for item in name_list:
    bands_file = output + item  + '_selected_bands.pkl'
    f1_score_file = output + item + '_selected_bands_f1_scores.pkl'
    
    _bands = pickle.load(open(bands_file,'rb'))
    print(item )
    print(_bands)
    _f1_score = pickle.load(open(f1_score_file, 'rb'))
    print(np.mean(_f1_score, axis=0))
    print()



H vs. 1-15
['402.8', '411', '764.1', '739.5', '721', '774.4', '811.3', '690.2', '682']
[0.8849232  0.90458042]

H vs. Sc35-70
['760', '714.8', '710.7', '772.3', '782.6', '780.5', '860.6', '776.4']
[0.9449225  0.91598973]

H vs. Sc 16-34
['762.1', '749.7', '404.9', '421.3', '409', '415.1']
[0.99808934 0.99883251]

Multiclasses
['402.8', '760', '725.1', '716.9', '411', '817.5', '766.2', '419.2', '809.3', '692.3', '622.5', '497.2', '677.9', '445.9', '509.6', '612.2', '663.5', '431.6', '456.2', '755.9', '706.6', '647.1']
[0.82095407 0.79283137 0.9985118  0.88146318]

spss Veg class1
['Chl green ', 'GNDVI.1', 'NDVI 761', 'PRI', 'NPQI', 'RVSI', 'SIPI', 'CVI', 'RARSc', 'CI green', 'GNDVI']
[0.82014097 0.83249737]

Spss veg class 2
['RVSI', 'Chl green ', 'GNDVI.1', 'NDVI 705', 'SIPI', 'RARSa']
[0.99663192 0.99679701]

Spss veg class 3
['CI rededge 710', 'RVSI', 'OSAVI', 'PSSRa', 'NDVI 780', 'NDVI 761']
[0.90655784 0.89639186]

SPSS multi VEG adjusted
['PRI', 'SIPI', 'RARSa', 'NDVI 761', 'ND 75

In [107]:
sheet_list = ['Vge H vs. Vege Class 2','Vege H vs. Class 3']

In [110]:
for sheet_name in sheet_list:
    bands_file = output + item  + '_selected_veg_index.pkl'
    _bands = pickle.load(open(bands_file,'rb'))
    print(sheet_name)
    _input, label = process_data(file_name, sheet_name)
    process_modes_bands(_input, label, _bands)

Vge H vs. Vege Class 2
SVC [0.88360031 0.882842  ]
DecisionTreeClassifier [0.82968289 0.83396414]
RandomForestClassifier [0.88174489 0.88608906]
Gaussian process classification [0.85147831 0.85382488]
AdaBoostClassifier [0.68030552 0.67504457]
KNeighborsClassifier [0.73675255 0.77463632]
Gaussian Naive Bayes [0.87516374 0.88193785]
Vege H vs. Class 3
SVC [0.94874364 0.93712706]
DecisionTreeClassifier [0.93400072 0.92362788]
RandomForestClassifier [0.94962715 0.94061897]
Gaussian process classification [0.92667875 0.91294971]
AdaBoostClassifier [0.86930091 0.84461178]
KNeighborsClassifier [0.85241368 0.82916879]
Gaussian Naive Bayes [0.95455369 0.94584924]


### Best Six bands

In [9]:
output = './output/'
#name_list = ['Multi classes', 'Veg H vs. Vge Class 1','Vge H vs. Vege Class 2','Vege H vs. Class 3']
print("Results for Best Six bands")
for item in name_list:
    bands_6_file = output + item  + '_6_bands.pkl'
    f1_score_6_file = output + item + '_6_bands_f1_scores.pkl'
    if not os.path.isfile(bands_6_file):
        continue
    
    _bands = pickle.load(open(bands_6_file,'rb'))
    print(item )
    print(_bands)
    _f1_score = pickle.load(open(f1_score_6_file, 'rb'))
    print(np.mean(_f1_score, axis=0))
    print()

Results for Best Six bands
H vs. 1-15
{'811.3', '690.2', '764.1', '774.4', '739.5', '682'}
[0.89561886 0.90667285]

H vs. Sc35-70
{'860.6', '714.8', '776.4', '710.7', '760', '782.6'}
[0.95128086 0.931072  ]

Multiclasses
{'817.5', '612.2', '760', '725.1', '497.2', '706.6'}
[0.79590929 0.77454589 0.99354534 0.86524802]

spss Veg class1
{'GNDVI.1', 'PRI', 'RARSc', 'Chl green ', 'SIPI', 'RVSI'}
[0.81756291 0.83973332]

SPSS multi VEG adjusted
{'GNDVI', 'PRI', 'Chl green ', 'NPQI', 'RVSI', 'BWDRVI'}
[0.77670529 0.7654231  0.99685447 0.85711111]



In [98]:
results[-1]

{'score': 0.8589094485320901,
 'bands': ['Chl green ',
  'GNDVI.1',
  'NDVI 780',
  'RDVI',
  'TVI',
  'RVSI',
  'PRI',
  'SR860',
  'PSSRa',
  'MCAR 1',
  'RARSc',
  'SIPI',
  'VARI',
  'ARI']}

## To get each method F1 score base on selected bands

In [15]:
file_name = 'Adjusted categorization - 1-5,16-30, 35-70.xlsx'
xl = pd.ExcelFile(file_name)
name_list = xl.sheet_names
name_list.remove('All categories')
name_list.remove('VEG index')

In [111]:
estimators = [
    ('RandomForestClassifier', RandomForestClassifier(n_estimators=10, random_state=42)),
    ('SVC', SVC(kernel = 'rbf')),
    ('KNeighborsClassifier',KNeighborsClassifier(n_neighbors=3)),
    ('GaussianNB',GaussianNB()),
    ('QuadraticDiscriminantAnalysis',QuadraticDiscriminantAnalysis()),
    ('DecisionTreeClassifier',DecisionTreeClassifier())
]

In [131]:
def get_scores_bands(X, y,sheet_name,estimators):

    #print("#######################################################")
    t1 = time()
    rkf = RepeatedKFold(n_splits=4, n_repeats=4, random_state=2652124)
    scores = list()
    for item in estimators:
        name = item[0]
        clf  = item[1]        
        f1_scores = []
        for train_index, test_index in rkf.split(X):
            #print("TRAIN:", train_index, "TEST:", test_index)
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]


            clf.fit(X_train,y_train)
            y_pred = clf.predict(X_test)
            _score = f1_score(y_test, y_pred, average=None)
            f1_scores.append(_score)

        f1_scores = np.asarray(f1_scores)
        #print(name, np.mean(f1_scores, axis=0))
        #scores.append(np.mean(f1_scores,axis=0))
        scores.append(np.mean(f1_scores))
        
    return name, scores

In [113]:
def process_data(file_name, sheet_name, estimators):
    full_data = pd.read_excel(file_name, sheet_name= sheet_name)

    str_cols = full_data.columns.astype(str)
    full_data.columns = str_cols
    cols = str_cols.tolist()
    cols = cols[1:]

    _input = full_data[cols]
    label = full_data['label'].tolist()
    label = np.asarray(label).reshape(-1,)
    print(sheet_name)
    print("Process selected best bands")
    bands_file = output + sheet_name  + '_selected_bands.pkl'
    selected_bands = pickle.load(open(bands_file,'rb'))
    print('Selected bands: {}'.format(selected_bands))
    
    X = _input[selected_bands].to_numpy()
    y = label
    get_scores_bands(X, y,sheet_name,estimators)
    
    print()

    bands_file = output + sheet_name  + '_6_bands.pkl'
    if os.path.isfile(bands_file):
        print("Process best Six bands")
        selected_bands = pickle.load(open(bands_file,'rb'))
        print('Best Six bands: {}'.format(selected_bands))

        X = _input[selected_bands].to_numpy()
        y = label
        get_scores_bands(X, y,sheet_name,estimators)

In [47]:
for sheet_name in name_list:
    process_data(file_name, sheet_name, estimators)
    print()

H vs. 1-15
Process selected best bands
Selected bands: ['402.8', '411', '764.1', '739.5', '721', '774.4', '811.3', '690.2', '682']
RandomForestClassifier [0.80012968 0.81526392]
SVC [0.66733554 0.74044267]
KNeighborsClassifier [0.77915938 0.81071426]
GaussianNB [0.65007013 0.6416217 ]
QuadraticDiscriminantAnalysis [0.90211655 0.9027479 ]
DecisionTreeClassifier [0.77261219 0.8032632 ]

Process best Six bands
Best Six bands: {'811.3', '690.2', '764.1', '774.4', '739.5', '682'}
RandomForestClassifier [0.78570126 0.80176645]
SVC [0.68748326 0.75823745]
KNeighborsClassifier [0.7834687  0.81682976]
GaussianNB [0.61614021 0.61058405]
QuadraticDiscriminantAnalysis [0.90037772 0.90820486]
DecisionTreeClassifier [0.78385586 0.81603361]

H vs. Sc35-70
Process selected best bands
Selected bands: ['760', '714.8', '710.7', '772.3', '782.6', '780.5', '860.6', '776.4']
RandomForestClassifier [0.90095546 0.8572297 ]
SVC [0.87616663 0.80252504]
KNeighborsClassifier [0.895184   0.85424521]
GaussianNB [0.

## Get each Veg Index  F1 Score

In [145]:
file_name = 'Adjusted categorization - 1-5,16-30, 35-70.xlsx'
sheet_list = ['spss Veg class1', 'Spss veg class 2', 'Spss veg class 3', 'SPSS multi VEG adjusted']

In [127]:
def cal_each_veg_index_scores(file_name, sheet_name, estimators):
    
    full_data = pd.read_excel(file_name, sheet_name= sheet_name)

    str_cols = full_data.columns.astype(str)
    full_data.columns = str_cols
    cols = str_cols.tolist()
    cols = cols[1:]

    _input = full_data[cols]
    label = full_data['label'].tolist()
    label = np.asarray(label).reshape(-1,)
    print(sheet_name)
    
    results = dict()
    for col in cols:
        X = _input[col].to_numpy().reshape(-1,1)
        y = label
        model,scores = get_scores_bands(X, y,sheet_name,estimators)
        results[col] = scores
        
        
    
    return results

In [135]:
_name = [item[0] for item in estimators]
_name

['RandomForestClassifier',
 'SVC',
 'KNeighborsClassifier',
 'GaussianNB',
 'QuadraticDiscriminantAnalysis',
 'DecisionTreeClassifier']

In [146]:
for sheet_name in sheet_list:
    results = cal_each_veg_index_scores(file_name, sheet_name, estimators)
    results['Model'] = _name
    r_pd = pd.DataFrame(results)
    r_file_name = 'Veg Index '+ sheet_name +' result.xlsx'
    r_pd.to_excel(r_file_name)


spss Veg class1
Spss veg class 2
Spss veg class 3
SPSS multi VEG adjusted


In [136]:
results['Model'] = _name

In [137]:
r_pd = pd.DataFrame(results)
r_pd

,RARSa,RARSb,RARSc,PSSRa,NDVI 780,NDVI 705,GNDVI,PRI,SR860,SIPI (Car/Chl),...,Chl green,CI green,CI rededge 710,CVI,BWDRVI,BRI,SIPI,OSAVI,VARI,Model
0,0.626621,0.643128,0.688376,0.639710,0.671901,0.650892,0.691956,0.643919,0.673986,0.644749,...,0.694012,0.685953,0.694882,0.686262,0.670020,0.658818,0.692043,0.649753,0.644425,RandomForestClassifier
1,0.550776,0.592267,0.648776,0.616196,0.617553,0.556397,0.668982,0.504475,0.617499,0.592805,...,0.692296,0.665348,0.630582,0.643434,0.625726,0.578605,0.629162,0.588006,0.523992,SVC
2,0.571037,0.594504,0.636464,0.581360,0.595865,0.585961,0.642139,0.572355,0.620671,0.576994,...,0.658646,0.647198,0.639414,0.617444,0.629918,0.596623,0.643253,0.604675,0.573427,KNeighborsClassifier
3,0.556383,0.581131,0.603093,0.577903,0.614647,0.564201,0.666786,0.503013,0.578567,0.579821,...,0.693660,0.622394,0.599400,0.630823,0.629477,0.540697,0.618548,0.606056,0.555387,GaussianNB
4,0.556383,0.581155,0.603044,0.577605,0.614647,0.563969,0.666786,0.503013,0.578713,0.579623,...,0.693660,0.622545,0.599400,0.630914,0.629460,0.540697,0.618349,0.606221,0.555356,QuadraticDiscriminantAnalysis
5,0.651510,0.653642,0.701352,0.656955,0.688355,0.662940,0.705337,0.661813,0.688474,0.661318,...,0.706418,0.697391,0.706957,0.696211,0.683576,0.672599,0.704012,0.668271,0.658080,DecisionTreeClassifier


In [1]:
import pickle

In [2]:
_dir = './output/'

In [14]:
clf = QuadraticDiscriminantAnalysis()
cv = RepeatedKFold(n_splits=4,n_repeats=4, random_state=46)

for i in range(1,17):
    file_name = _dir + "comb_6_"+ str(i) + '_bands.pkl'
    _bands = pickle.load(open(file_name, 'rb'))
    _new_input = _input[_bands]
    scores = cross_val_score(clf, _new_input, label, scoring='accuracy', cv=cv, n_jobs=-1)
    _score = np.mean(scores)
    print(_bands, " : ", _score)

['813.4', '747.7', '737.4', '721', '774.4', '803.1']  :  0.9077939462373426
['813.4', '737.4', '762.1', '801.1', '706.6', '774.4']  :  0.908970914985066
['396.7', '737.4', '778.5', '762.1', '706.6', '774.4']  :  0.8936270670940483
['396.7', '731.3', '721', '757.9', '766.2', '774.4']  :  0.9034107051795731
['827.7', '737.4', '731.3', '774.4', '875', '803.1']  :  0.9023999235084141
['671.7', '737.4', '778.5', '762.1', '774.4', '696.4']  :  0.9005436366285423
['562.9', '737.4', '757.9', '772.3', '768.2', '538.3']  :  0.9081308734610621
['562.9', '749.7', '745.6', '766.2', '768.2', '538.3']  :  0.8899258760107818
['747.7', '737.4', '721', '801.1', '774.4', '875']  :  0.9081322393822393
['747.7', '737.4', '778.5', '776.4', '764.1', '706.6']  :  0.9023944598237051
['737.4', '811.3', '762.1', '706.6', '774.4', '803.1']  :  0.9104916405623953
['737.4', '778.5', '762.1', '774.4', '696.4', '675.8']  :  0.9039142747869162
['811.3', '731.3', '721', '762.1', '774.4', '803.1']  :  0.9099844284985794

In [54]:
rfe_svc = set(['725.1', '762.1', '723', '587.6', '733.3', '421.3', '846.2', '776.4', '817.5', '406.9', '774.4', 
           '413.1', '499.3', '458.2', '517.8', '597.8', '864.7', '805.2', '614.2'])
rfe_random = set(['411', '757.9', '772.3', '727.2', '721', '817.5', '511.6', '484.9', '577.3', '860.6', '445.9', '521.9', 
              '417.2', '655.3', '398.7', '803.1', '536.2', '454.1', '556.8', '751.8', 
              '673.8', '743.6', '833.9', '688.1', '883.2', '630.7', '406.9', '429.5', '468.5'])
tree = set(['402.8', '760', '764.1', '714.8', '819.5', '721', '415.1', '513.7', '495.2', '433.6', '618.4', 
        '445.9', '651.2', '640.9', '679.9', '694.3', '409', '423.3', '706.6', '411', '661.5', '669.7'])
R_regression = set(['677.9', '762.1', '745.6', '731.3', '415.1', '723', '606', '716.9', '622.5', '877', '659.4', 
                '643', '497.2', '472.6', '825.7', '454.1', '573.2', '402.8', '696.4', '776.4', '550.6', '513.7', '530.1', '411'])

RidgeClassifier = set(['764.1', '770.3', '723', '827.7', '718.9', '589.6', '411', '735.4', '805.2', '400.8', '872.9', '792.8', '817.5', 
                   '784.6', '571.1', '692.3', '636.8', '515.7', '466.4', '679.9', '528', '495.2', '433.6', '651.2', '751.8', '421.3', 
                   '745.6', '552.7', '885.2', '406.9', '846.2'])

lasso = set(['885.2', '817.5', '749.7', '558.8', '757.9', '400.8', '421.3', '507.5', '825.7', '445.9', '731.3', '721', '858.5', 
         '657.4', '542.4', '521.9', '692.3', '577.3', '741.5', '409', '478.8'])

chi2 = set(['665.6', '762.1', '735.4', '721', '842.1', '606', '696.4', '774.4', '409', '419.2', '509.6', '482.9', '591.7', '885.2', 
        '792.8', '523.9', '445.9', '536.2', '776.4', '801.1', '556.8', '624.5'])

f_classif = set(['677.9', '762.1', '745.6', '731.3', '415.1', '723', '606', '716.9', '622.5', '877', '659.4', '643', '497.2', '472.6', 
             '825.7', '454.1', '573.2', '402.8', '696.4', '776.4', '550.6', '513.7', '530.1', '411'])

In [55]:
full_lists = set.union(rfe_svc, rfe_random, tree, R_regression, RidgeClassifier, lasso, chi2, f_classif )

full_lists = list(full_lists) 

In [60]:
full_lists.sort()

In [61]:
full_lists


['398.7',
 '400.8',
 '402.8',
 '406.9',
 '409',
 '411',
 '413.1',
 '415.1',
 '417.2',
 '419.2',
 '421.3',
 '423.3',
 '429.5',
 '433.6',
 '445.9',
 '454.1',
 '458.2',
 '466.4',
 '468.5',
 '472.6',
 '478.8',
 '482.9',
 '484.9',
 '495.2',
 '497.2',
 '499.3',
 '507.5',
 '509.6',
 '511.6',
 '513.7',
 '515.7',
 '517.8',
 '521.9',
 '523.9',
 '528',
 '530.1',
 '536.2',
 '542.4',
 '550.6',
 '552.7',
 '556.8',
 '558.8',
 '571.1',
 '573.2',
 '577.3',
 '587.6',
 '589.6',
 '591.7',
 '597.8',
 '606',
 '614.2',
 '618.4',
 '622.5',
 '624.5',
 '630.7',
 '636.8',
 '640.9',
 '643',
 '651.2',
 '655.3',
 '657.4',
 '659.4',
 '661.5',
 '665.6',
 '669.7',
 '673.8',
 '677.9',
 '679.9',
 '688.1',
 '692.3',
 '694.3',
 '696.4',
 '706.6',
 '714.8',
 '716.9',
 '718.9',
 '721',
 '723',
 '725.1',
 '727.2',
 '731.3',
 '733.3',
 '735.4',
 '741.5',
 '743.6',
 '745.6',
 '749.7',
 '751.8',
 '757.9',
 '760',
 '762.1',
 '764.1',
 '770.3',
 '772.3',
 '774.4',
 '776.4',
 '784.6',
 '792.8',
 '801.1',
 '803.1',
 '805.2',
 '817.

In [ ]:
2,813,729,380